#########################################
##      뉴스 테마 및 테마주 크롤링      ##
#########################################

0. 라이브러리 임포트

In [1]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict
import pickle

1. 변수들 선언

In [2]:
page = 0
theme_list = []
source_list = []
theme_dict = defaultdict(list)

2. 크롤링 막힘 방지용 헤더 설정

In [3]:
# User-Agent 설정
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'ko-KR,ko;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive'
}

3. 테마 및 테마주 크롤링

In [4]:
for i in range(0, 8):
  page += 1
  url = f'https://finance.naver.com/sise/theme.naver?&page={page}'
  response = requests.get(url, headers=headers)                                 #   네이버 테마 사이트

  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    theme_elements = soup.find_all(class_='col_type1')
    for theme_element in theme_elements:
      a_tag = theme_element.find('a')
      if a_tag and a_tag.get_text() != '테마명':
        stock_list = []
        theme_name = a_tag.get_text()                                           #   테마 이름
        theme_href = a_tag['href']                                              #   테마별 하이퍼링크
        source = ('https://finance.naver.com'+theme_href)
        stock_response = requests.get(source, headers=headers)
        stock_soup = BeautifulSoup(stock_response.text, 'html.parser')
        stock_elements = stock_soup.find_all(class_='name_area')                #   테마주 가져오기
        cnt = 0
        for stock in stock_elements:
            stock_text = stock.get_text()
            cleaned_text = re.sub(r'[^가-힣a-zA-Z0-9]', '', stock_text)
            theme_dict[theme_name].append(cleaned_text)                         #   딕셔너리 key(테마):value(테마주)
            cnt += 1
            if cnt == 5:                                                        #   최대 다섯개 테마주만 가져오기
              break

print(dict(theme_dict))

{'코로나19(렘데시비르)': ['파미셀', '에스티팜', '비씨월드제약'], '엠폭스(원숭이두창)': ['파미셀', '케스피온', '미코바이오메드', '녹십자엠에스', '제놀루션'], '화이자(PFIZER)': ['파미셀', '그린생명과학', '유한양행', '대한약품', '서울제약'], '모더나(MODERNA)': ['파미셀', '에스티팜', '소마젠', '삼성바이오로직스', '녹십자'], '코로나19(진단키트)': ['파미셀', '미코바이오메드', '바이오노트', '녹십자엠에스', '제놀루션'], '코로나19(음압병실/음압구급차)': ['우정바이오', 'GH신소재', '에스와이', '케이엔솔', '오텍'], '메르스 코로나 바이러스': ['우정바이오', '웰크론', '빌리언스', '케이엠', '진매트릭스'], '코로나19(스푸트니크V)': ['프레스티지바이오로직스', '프레스티지바이오파마', '이수앱지스', '보령', '제테마'], '마스크': ['웰크론', '녹십자엠에스', '모나리자', '깨끗한나라', '케이엠'], '황사/미세먼지': ['케스피온', '웰크론', '모나리자', '케이엠', '성창오토텍'], '반도체 대표주(생산)': ['SK하이닉스', '삼성전자', 'DB하이텍'], 'mRNA(메신저 리보핵산)': ['파미셀', '올리패스', '피씨엘', '아이진', '에스티팜'], 'HBM(고대역폭메모리)': ['테크윙', '에스티아이', '케이씨텍', '한미반도체', '이오테크닉스'], '페라이트': ['케스피온', '아모그린텍', '삼화전기', '씨큐브', '알엔투테크놀로지'], '미용기기': ['라메디텍', '한국비엔씨', '유한양행', '메디톡스', '휴메딕스'], '코로나19(진단/치료제/백신 개발 등)': ['케이바이오', '파미셀', '미코바이오메드', '바이오노트', '녹십자엠에스'], '선박평형수 처리장치': ['STX엔진', '한라IMS', '부방', 'HD한국조선해양', '엔케이'], '3D 낸드(NAND)': ['테크윙', '케

4. 이상한 테마 지우기

In [5]:
print(f'삭제 전 딕셔너리 길이 : {len(theme_dict)}')
keys_to_delete = [key for key in theme_dict if "신규상장" in key]

for key in keys_to_delete:
    del theme_dict[key]
print(f'삭제 후 딕셔너리 길이 : {len(theme_dict)}')

삭제 전 딕셔너리 길이 : 290
삭제 후 딕셔너리 길이 : 278


5. 피클로 저장

In [7]:
with open('D:/TJ_FInal_Project/KDJ/News_Keyword_Theme_Recommend.py/data/theme_dict.pkl', 'wb') as f:
    pickle.dump(theme_dict, f)
print(f'저장완료!')

FileNotFoundError: [Errno 2] No such file or directory: 'D:/TJ_FInal_Project/KDJ/New_Keyword_Theme_Recommend.py/data/theme_dict.pkl'